In [1]:
using Pkg
using LinearAlgebra

In [2]:
using TSSOS
using DynamicPolynomials
using Random
using Statistics
using DataFrames
using DelimitedFiles

In [3]:
function observation(T,ts,start)
    # randomly select a trajectory of time series
    return copy(ts[start:start+T-1])
end

observation (generic function with 1 method)

In [7]:
A = [[1, 2], [3,4]]
println(A[1])

[1, 2]


In [30]:
function parameter_estimation2(Y)
    # training process
    
    T=length(Y)
    measurements = length()
    
    @polyvar G[1:4] Fdash[1:2] p[1:T] q[1:2*T] f[1:T] m[1:2*(T+1)];
    var=vcat(G,Fdash,m,f,p,q);

    # constraints
    ine1 = [f[i] - Fdash[1]*m[2*i+1] - Fdash[2]*m[2*i+2] - p[i] for i in 1:T];
    ine2 = [-f[i] + Fdash[1]*m[2*i+1] + Fdash[2]*m[2*i+2] + p[i] for i in 1:T];
    ine3 = [m[i+2] - G[1]*m[i] - G[2]*m[i+1] - q[i]  for i in 1:2:2*T];
    ine4 = [-m[i+2] + G[1]*m[i] + G[2]*m[i+1] + q[i]  for i in 1:2:2*T];
    ine5 = [m[i+2] - G[3]*m[i-1] - G[4]*m[i] - q[i]  for i in 2:2:2*T];
    ine6 = [-m[i+2] + G[3]*m[i-1] + G[4]*m[i] + q[i]  for i in 2:2:2*T];

    #objective
    obj=sum(sum((Y[j][i]-f[i])^2 for i in 1:T) for j + 0.005*sum(p[i]^2 for i in 1:T) +0.001*sum(q[i]^2 for i in 1:T-1)

    # pop
    pop=vcat(obj,ine1,ine2,ine3,ine4,ine5,ine6);

    # solve model 
    opt,sol,data=tssos_first(pop,var,4,TS="MD",solution=true);
    
    println("sol: ", sol)
    println("opt: ", opt)
    #println("data: ", data)
    return opt, sol, data 
end

parameter_estimation2 (generic function with 1 method)

In [14]:
function rmse(Y_predict, Y_true)
    # compare predicted Y_[T+1：T+pred] with actual value of predicted Y_[T+1：T+pred]
    # calcluate rmse: sqrt( sum((Y-f)^2)/N) 
    Y=copy(Y_true[1:length(Y_predict)])
    return  abs(Y[1]-Y_predict[1])
    #1-sum( abs(Y[i]-Y_predict[i]) for i in 1:length(Y_predict) )/ sum(abs(Y[i]-mean(Y)) for i in 1:length(Y_predict) )
end
    

rmse (generic function with 1 method)

In [15]:
#read Hazan data
ts = readdlm( "Hazan_25_ts.txt" );

In [ ]:
## a single run (because the first time using TSSOS is very slow)
T=2 # the length of time window is 20
pred=1 # we are making one-step ahead prediction
Y=observation(T+pred,ts,1) # select a 21-period time series from stock-market data. 
opt, sol, data = parameter_estimation2(Y[1:T]) # system identification using the first 20-period

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[231, 21, 3, 2, 1]
[1, 210, 8, 20, 5985]
-----------------------------------------------------------------------------
Obtained the block structure in 208.128659769 seconds. The maximal size of blocks is 231.
Assembling the SDP...
There are 64672 affine constraints.
SDP assembling time: 11.990261177 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 64672           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 10354          

In [21]:
#for T=10 only
f5 = sol[5]*sol[17] + sol[6]*sol[18] + sol[43]
f5real = sol[33]
println(f5)
println(f5real)

1.2241642802709534
1.2241642804121389
